# ETL Project - TRANSFORM
## Terrence Cummings
Data: Race mix of Minneapolis neighborhoods scraped from mncompass.org

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import time
import sys
import os
import datetime
from selenium.webdriver.chrome.options import Options
from sqlalchemy import create_engine
chrome_options = Options()
chrome_options.add_argument("--headless")

In [2]:
#Read in CSV's from Extract
cmnty_links_df=pd.read_csv('cmnty_links.csv')
nbhd_links_df=pd.read_csv('nbhd_links.csv')
cmnty_race_df=pd.read_csv('cmnty_race.csv')
nbhd_race_df=pd.read_csv('nbhd_race.csv')
cmnty_keys_df=pd.read_csv('cmnty_keys.csv')
nbhd_keys_df=pd.read_csv('nbhd_keys.csv')

In [3]:
#Add community key to neighborhood key table to use as a foreign key
nbhd_keys_df = nbhd_keys_df.merge(cmnty_keys_df, how='left', left_on='COMMUNITY', right_on='COMMUNITY')


## Create initial table of MSP neighborhoods (rows) and race mix (columns)

In [4]:
#Some figures from the scraped table show the word 'suppressed'. Replace with NaN for subsequent handling.
nbhd_race_df = nbhd_race_df.replace('suppressed', np.nan)


In [5]:
#Convert population count text to numbers
nbhd_race_df['white_cnt'] = nbhd_race_df['white_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['black_cnt'] = nbhd_race_df['black_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['native_cnt'] = nbhd_race_df['native_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['asian_cnt'] = nbhd_race_df['asian_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['other_cnt'] = nbhd_race_df['other_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['two_or_more_cnt'] = nbhd_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['hispanic_cnt'] = nbhd_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
nbhd_race_df['of_color_cnt'] = nbhd_race_df['of_color_cnt'].str.replace(',', '').astype(float)

In [6]:
#Convert population percentage text to numbers
nbhd_race_df['white_pct'] = nbhd_race_df['white_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['black_pct'] = nbhd_race_df['black_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['native_pct'] = nbhd_race_df['native_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['asian_pct'] = nbhd_race_df['asian_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['other_pct'] = nbhd_race_df['other_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['two_or_more_pct'] = nbhd_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['hispanic_pct'] = nbhd_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
nbhd_race_df['of_color_pct'] = nbhd_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [7]:
#Drop ECCO neighborhood due to no data
nbhd_race_df.drop([11], inplace=True)

In [8]:
nbhd_race_df.loc[nbhd_race_df['of_color_cnt'].isnull(),'of_color_cnt'] = nbhd_race_df['white_cnt']/nbhd_race_df['white_pct']-nbhd_race_df['white_cnt']

In [9]:
#Add total population estimate for each neighborood based on the white count and percentage.

nbhd_race_df['total_cnt']=nbhd_race_df['white_cnt']+nbhd_race_df['of_color_cnt']
nbhd_race_df['of_color_pct']=nbhd_race_df['of_color_cnt']/nbhd_race_df['total_cnt']

In [10]:
#Get rid of counts. Not needed when have Total Count and percentages
nbhd_race_df=nbhd_race_df.drop(['white_cnt', 'black_cnt', 'native_cnt', 'asian_cnt','other_cnt','two_or_more_cnt','hispanic_cnt','of_color_cnt','URL'], axis = 1) 

#Add the neighborhood and community keys
nbhd_race_df = nbhd_race_df.merge(nbhd_keys_df, left_on='neighborhood', right_on='neighborhood_url')


In [11]:
#Reorder columns for presentability. At this stage have a table with all neighborhoods and races, but with a lot of NaN's that needed to be reconciled.
orig_nbhd_race_df = nbhd_race_df[['NBHD_KEY','total_cnt','white_pct','black_pct','native_pct','asian_pct','other_pct','two_or_more_pct','hispanic_pct','of_color_pct']]

orig_nbhd_race_df.rename(columns = {'NBHD_KEY':'neighborhood_id'}, inplace = True) 

final_nbhd_race_df=orig_nbhd_race_df

## Create table of MSP communities (rows) and race mix (columns). This will be used to backfill missing neighborhood-level data.

In [12]:
#Set the few 'suppressed' data to 0 as immaterial. 
cmnty_race_df['community'] = cmnty_race_df['community'].str.upper()
cmnty_race_df = cmnty_race_df.replace('suppressed', np.nan)
#cmnty_race_df = cmnty_race_df.replace(np.nan, '0', regex=True)


In [13]:
#Convert race population count text to numbers.
cmnty_race_df['white_cnt'] = cmnty_race_df['white_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['black_cnt'] = cmnty_race_df['black_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['native_cnt'] = cmnty_race_df['native_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['asian_cnt'] = cmnty_race_df['asian_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['other_cnt'] = cmnty_race_df['other_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['two_or_more_cnt'] = cmnty_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['hispanic_cnt'] = cmnty_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['of_color_cnt'] = cmnty_race_df['of_color_cnt'].str.replace(',', '').astype(float)


In [14]:
#Convert race population percentages to numbers
cmnty_race_df['white_pct'] = cmnty_race_df['white_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['black_pct'] = cmnty_race_df['black_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['native_pct'] = cmnty_race_df['native_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['asian_pct'] = cmnty_race_df['asian_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['other_pct'] = cmnty_race_df['other_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['two_or_more_pct'] = cmnty_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['hispanic_pct'] = cmnty_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['of_color_pct'] = cmnty_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [15]:
#Create a dicationary to map the scraped community names to the official community names
cmnty_dict = {'CALHOUN-ISLES': 'CALHOUN-ISLE', 'CAMDEN':'CAMDEN', 'CENTRAL': 'CENTRAL', 'LONGFELLOW': 'LONGFELLOW', 'NEAR-NORTH': 'NEAR NORTH', 'NOKOMIS':'NOKOMIS', 'NORTHEAST':'NORTHEAST', 'PHILLIPS': 'PHILLIPS', 'POWDERHORN': 'POWDERHORN', 'SOUTHWEST':'SOUTHWEST', 'UNIVERSITY':'UNIVERSITY'}

In [16]:
#Replaced scraped community names with official community names
cmnty_race_df = cmnty_race_df.replace({"community": cmnty_dict})

In [17]:
#Add community keys to the community race table
cmnty_race_df = cmnty_race_df.merge(cmnty_keys_df, how='left', left_on='community', right_on='COMMUNITY')
del cmnty_race_df['COMMUNITY']

#Add total population estimate for each community based on the white count and percentage.
cmnty_race_df['total_cnt']=round(cmnty_race_df['white_cnt']/cmnty_race_df['white_pct'],0)

#Drop unnecssary columns
cmnty_race_df=cmnty_race_df.drop(['white_cnt', 'black_cnt', 'native_cnt', 'asian_cnt','other_cnt','two_or_more_cnt','hispanic_cnt','of_color_cnt','URL'], axis = 1)

In [18]:
#Cleanup column order and names
orig_cmnty_race_df = cmnty_race_df[['CMNTY_KEY','total_cnt','white_pct','black_pct','native_pct','asian_pct','other_pct','two_or_more_pct','hispanic_pct','of_color_pct']]

orig_cmnty_race_df.rename(columns = {'CMNTY_KEY':'community_id'}, inplace = True) 
final_cmnty_race_df=orig_cmnty_race_df


## CREATE FINAL TABLES AND LOAD INTO POSTGRESQL


In [19]:
#Write the final neighborhood and community race mix tables to csv's
final_nbhd_race_df.to_csv('../target_files/nbhd_race_sql.csv')
final_cmnty_race_df.to_csv('../target_files/cmnty_race_sql.csv')


In [20]:
host = "localhost"
user = "postgres"
port = "5432"
passwd = "hawkeyes"
db = "REP_13_ETL_Project"

engine = create_engine(f'postgresql://{user}:{passwd}@{host}:{port}/{db}')
connection = engine.connect()

In [21]:
communities_df=pd.read_csv('../target_files/MLPS_Communities.csv')
neighorhoods_df=pd.read_csv('../target_files/MLPS_Neighborhoods.csv')

In [22]:
final_nbhd_race_df.to_sql('NEIGHBORHOOD_RACE', con=connection, if_exists = 'append', index=False)
final_cmnty_race_df.to_sql('COMMUNITY_RACE', con=connection, if_exists = 'append', index=False)
communities_df.to_sql('COMMUNITY', con=connection, if_exists = 'append', index=False)
neighorhoods_df.to_sql('NEIGHBORHOOD',con=connection, if_exists = 'append', index=False)